In [145]:
import yfinance as yf
import pandas as pd
import ta
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
import joblib


In [146]:
tickers = {
    "NIFTY": "^NSEI",
    "BANKNIFTY": "^NSEBANK"
}

intervals = {
    "1m": "7d",
    "5m": "60d",
    "15m": "60d",
    "30m": "60d",
    "60m": "60d"
}


In [147]:
os.makedirs("nifty_banknifty_raw", exist_ok=True)

def fetch_raw_data(ticker_symbol, interval, period):
    df = yf.download(ticker_symbol, interval=interval, period=period)
    df.dropna(inplace=True)
    return df

for name, ticker in tickers.items():
    for interval, period in intervals.items():
        df = fetch_raw_data(ticker, interval, period)
        path = f"nifty_banknifty_raw/{name}_{interval}_raw.csv"
        df.to_csv(path)
        print(f"✅ Saved: {path}")


C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\3630785718.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker_symbol, interval=interval, period=period)
[*********************100%***********************]  1 of 1 completed
C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\3630785718.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker_symbol, interval=interval, period=period)


✅ Saved: nifty_banknifty_raw/NIFTY_1m_raw.csv


[*********************100%***********************]  1 of 1 completed
C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\3630785718.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker_symbol, interval=interval, period=period)


✅ Saved: nifty_banknifty_raw/NIFTY_5m_raw.csv


[*********************100%***********************]  1 of 1 completed
C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\3630785718.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker_symbol, interval=interval, period=period)


...

✅ Saved: nifty_banknifty_raw/BANKNIFTY_15m_raw.csv


[*********************100%***********************]  1 of 1 completed
C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\3630785718.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker_symbol, interval=interval, period=period)


✅ Saved: nifty_banknifty_raw/BANKNIFTY_30m_raw.csv


[*********************100%***********************]  1 of 1 completed

✅ Saved: nifty_banknifty_raw/BANKNIFTY_60m_raw.csv


In [148]:
def apply_feature_engineering(df):
    df = df.copy()

    # Ensure numeric format
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df.dropna(subset=['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)

    # Technical Indicators
    df['rsi'] = ta.momentum.RSIIndicator(close=df['Close']).rsi()
    macd = ta.trend.MACD(close=df['Close'])
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()
    df['ema_20'] = ta.trend.EMAIndicator(close=df['Close'], window=20).ema_indicator()
    df['ema_50'] = ta.trend.EMAIndicator(close=df['Close'], window=50).ema_indicator()
    df['atr'] = ta.volatility.AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close']).average_true_range()

    df.dropna(inplace=True)
    return df


In [149]:
os.makedirs("nifty_banknifty_processed", exist_ok=True)

for name in tickers:
    for interval in intervals:
        path = f"nifty_banknifty_raw/{name}_{interval}_raw.csv"
        df = pd.read_csv(path, index_col=0, parse_dates=True)
        df = apply_feature_engineering(df)
        df.to_csv(f"nifty_banknifty_processed/{name}_{interval}_processed.csv")
        print(f"✅ Processed and saved: {name}_{interval}_processed.csv")


C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\313334127.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, index_col=0, parse_dates=True)
C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\313334127.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, index_col=0, parse_dates=True)


✅ Processed and saved: NIFTY_1m_processed.csv


C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\313334127.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, index_col=0, parse_dates=True)


✅ Processed and saved: NIFTY_5m_processed.csv
✅ Processed and saved: NIFTY_15m_processed.csv
✅ Processed and saved: NIFTY_30m_processed.csv


C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\313334127.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, index_col=0, parse_dates=True)
C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\313334127.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, index_col=0, parse_dates=True)
C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\313334127.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, index_col=0, parse_dates=True)


✅ Processed and saved: NIFTY_60m_processed.csv
✅ Processed and saved: BANKNIFTY_1m_processed.csv


C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\313334127.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, index_col=0, parse_dates=True)


✅ Processed and saved: BANKNIFTY_5m_processed.csv
✅ Processed and saved: BANKNIFTY_15m_processed.csv
✅ Processed and saved: BANKNIFTY_30m_processed.csv


C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\313334127.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, index_col=0, parse_dates=True)
C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\313334127.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, index_col=0, parse_dates=True)
C:\Users\Jimit\AppData\Local\Temp\ipykernel_22424\313334127.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, index_col=0, parse_dates=True)


✅ Processed and saved: BANKNIFTY_60m_processed.csv


In [150]:
def add_target_labels(df, close_col, shift_period=1, threshold_pct=0.00085):
    df = df.copy()
    df['future_close'] = df[close_col].shift(-shift_period)
    df['return_pct'] = (df['future_close'] - df[close_col]) / df[close_col]
    df['target'] = 1
    df.loc[df['return_pct'] > threshold_pct, 'target'] = 2
    df.loc[df['return_pct'] < -threshold_pct, 'target'] = 0
    df.dropna(inplace=True)
    return df


In [151]:
os.makedirs("nifty_banknifty_labeled", exist_ok=True)
for name in tickers:
    for interval in intervals:
        file_path = f"nifty_banknifty_processed/{name}_{interval}_processed.csv"
        df = pd.read_csv(file_path, index_col=0, parse_dates=True)
        df = add_target_labels(df, close_col='Close')
        df.to_csv(f"nifty_banknifty_labeled/{name}_{interval}_labeled.csv")
        print(f"✅ Labeled: {name}_{interval}_labeled.csv")
        print(df['target'].value_counts(normalize=True))



✅ Labeled: NIFTY_1m_labeled.csv
target
1    0.990665
2    0.004667
0    0.004667
Name: proportion, dtype: float64
✅ Labeled: NIFTY_5m_labeled.csv
target
1    0.861798
2    0.069213
0    0.068989
Name: proportion, dtype: float64
✅ Labeled: NIFTY_15m_labeled.csv
target
1    0.685517
0    0.158621
2    0.155862
Name: proportion, dtype: float64
✅ Labeled: NIFTY_30m_labeled.csv
target
1    0.561194
0    0.240299
2    0.198507
Name: proportion, dtype: float64
✅ Labeled: NIFTY_60m_labeled.csv
target
1    0.470270
2    0.267568
0    0.262162
Name: proportion, dtype: float64
✅ Labeled: BANKNIFTY_1m_labeled.csv
target
1    0.993388
2    0.003501
0    0.003112
Name: proportion, dtype: float64
✅ Labeled: BANKNIFTY_5m_labeled.csv
target
1    0.833483
2    0.085618
0    0.080899
Name: proportion, dtype: float64
✅ Labeled: BANKNIFTY_15m_labeled.csv
target
1    0.641379
0    0.186207
2    0.172414
Name: proportion, dtype: float64
✅ Labeled: BANKNIFTY_30m_labeled.csv
target
1    0.495522
0    0.255224


In [152]:
import os
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import joblib

# Target intervals and symbols
intervals = ["5m", "15m", "30m", "60m"]
symbols = ["NIFTY", "BANKNIFTY"]

# Ensure model output directory exists
os.makedirs("final_models", exist_ok=True)

for symbol in symbols:
    for interval in intervals:
        print(f"\n📘 Training {symbol} - {interval} model...")

        # Load labeled dataset
        path = f"nifty_banknifty_labeled/{symbol}_{interval}_labeled.csv"
        if not os.path.exists(path):
            print(f"❌ File not found: {path}")
            continue
        df = pd.read_csv(path, index_col=0, parse_dates=True)

        # Drop helper columns if present
        df.drop(columns=[col for col in ['future_close', 'return_pct'] if col in df.columns], inplace=True)

        # Split features and target
        X = df.drop(columns=['target'])
        y = df['target']

        # Sanitize data
        for col in X.columns:
            if X[col].dtype == 'object':
                X[col] = pd.to_numeric(X[col], errors='coerce')
        X = X.select_dtypes(include=['number']).fillna(0)

        # Check class count
        if y.nunique() < 2:
            print(f"❌ Skipping {symbol}-{interval}: Only one class in data.")
            continue
        elif y.nunique() < 3:
            print(f"⚠️ Warning: Only classes {y.unique().tolist()} found in {symbol}-{interval}")

        # Stratified train-test split
        try:
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, stratify=y, random_state=42
            )
        except ValueError as e:
            print(f"⚠️ Skipping due to stratify error: {e}")
            continue

        # Train model
        model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
        model.fit(X_train, y_train)

        # Evaluate
        y_pred = model.predict(X_test)
        print("📊 Classification Report:")
        print(classification_report(y_test, y_pred))
        print("🔍 Confusion Matrix:")
        print(confusion_matrix(y_test, y_pred))

        # Save model
        model_path = f"final_models/{symbol}_model_{interval}.pkl"
        joblib.dump(model, model_path)
        print(f"✅ Model saved to {model_path}")



📘 Training NIFTY - 5m model...


c:\Users\Jimit\Desktop\Course 2\Sem 2\Case studies\AI_PREDICTOR\venv\lib\site-packages\xgboost\training.py:183: UserWarning: [06:16:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


📊 Classification Report:
              precision    recall  f1-score   support

           0       0.15      0.03      0.05        61
           1       0.87      0.98      0.92       767
           2       0.12      0.03      0.05        62

    accuracy                           0.85       890
   macro avg       0.38      0.35      0.34       890
weighted avg       0.77      0.85      0.80       890

🔍 Confusion Matrix:
[[  2  54   5]
 [  7 750  10]
 [  4  56   2]]
✅ Model saved to final_models/NIFTY_model_5m.pkl

📘 Training NIFTY - 15m model...


c:\Users\Jimit\Desktop\Course 2\Sem 2\Case studies\AI_PREDICTOR\venv\lib\site-packages\xgboost\training.py:183: UserWarning: [06:16:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


📊 Classification Report:
              precision    recall  f1-score   support

           0       0.29      0.22      0.25        46
           1       0.72      0.85      0.78       199
           2       0.25      0.11      0.15        45

    accuracy                           0.64       290
   macro avg       0.42      0.39      0.40       290
weighted avg       0.58      0.64      0.60       290

🔍 Confusion Matrix:
[[ 10  32   4]
 [ 18 170  11]
 [  6  34   5]]
✅ Model saved to final_models/NIFTY_model_15m.pkl

📘 Training NIFTY - 30m model...


...

📊 Classification Report:
              precision    recall  f1-score   support

           0       0.17      0.09      0.12        54
           1       0.65      0.81      0.72       186
           2       0.11      0.06      0.08        50

    accuracy                           0.54       290
   macro avg       0.31      0.32      0.30       290
weighted avg       0.46      0.54      0.50       290

🔍 Confusion Matrix:
[[  5  41   8]
 [ 19 150  17]
 [  6  41   3]]
✅ Model saved to final_models/BANKNIFTY_model_15m.pkl

📘 Training BANKNIFTY - 30m model...


c:\Users\Jimit\Desktop\Course 2\Sem 2\Case studies\AI_PREDICTOR\venv\lib\site-packages\xgboost\training.py:183: UserWarning: [06:17:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


📊 Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.24      0.28        34
           1       0.51      0.64      0.56        66
           2       0.30      0.24      0.26        34

    accuracy                           0.43       134
   macro avg       0.38      0.37      0.37       134
weighted avg       0.41      0.43      0.41       134

🔍 Confusion Matrix:
[[ 8 18  8]
 [13 42 11]
 [ 3 23  8]]
✅ Model saved to final_models/BANKNIFTY_model_30m.pkl

📘 Training BANKNIFTY - 60m model...


c:\Users\Jimit\Desktop\Course 2\Sem 2\Case studies\AI_PREDICTOR\venv\lib\site-packages\xgboost\training.py:183: UserWarning: [06:17:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


📊 Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.27      0.30        22
           1       0.46      0.59      0.52        32
           2       0.27      0.20      0.23        20

    accuracy                           0.39        74
   macro avg       0.35      0.36      0.35        74
weighted avg       0.37      0.39      0.38        74

🔍 Confusion Matrix:
[[ 6 11  5]
 [ 7 19  6]
 [ 5 11  4]]
✅ Model saved to final_models/BANKNIFTY_model_60m.pkl


In [153]:
import pandas as pd
import joblib
from collections import Counter

# Configuration
intervals = ["5m", "15m", "30m", "60m"]
symbols = ["NIFTY", "BANKNIFTY"]
model_dir = "final_models"

# Load the latest feature set (replace this with your actual input features)
# For testing, load last row from any labeled dataset
def load_latest_features(symbol, interval):
    path = f"nifty_banknifty_labeled/{symbol}_{interval}_labeled.csv"
    df = pd.read_csv(path, index_col=0, parse_dates=True)
    df.drop(columns=[col for col in ['target', 'future_close', 'return_pct'] if col in df.columns], inplace=True)
    df = df.select_dtypes(include='number').fillna(0)
    return df.iloc[[-1]]  # last row as latest input

# Voting prediction function
def get_final_signal(symbol):
    votes = []
    for interval in intervals:
        model_path = f"{model_dir}/{symbol}_model_{interval}.pkl"
        if not os.path.exists(model_path):
            print(f"❌ Missing model: {model_path}")
            continue
        
        model = joblib.load(model_path)
        latest_features = load_latest_features(symbol, interval)
        pred = model.predict(latest_features)[0]
        votes.append(pred)

    # Majority vote
    if not votes:
        return "❌ No predictions available"
    
    vote_count = Counter(votes)
    final_decision = vote_count.most_common(1)[0][0]
    
    signal_map = {0: "📉 Suggest PUT (PE)", 1: "⏸️ Suggest NO ACTION", 2: "📈 Suggest CALL (CE)"}
    return signal_map[final_decision]

# Example usage
print("📊 Final Decision for NIFTY:", get_final_signal("NIFTY"))
print("📊 Final Decision for BANKNIFTY:", get_final_signal("BANKNIFTY"))


📊 Final Decision for NIFTY: ⏸️ Suggest NO ACTION
📊 Final Decision for BANKNIFTY: ⏸️ Suggest NO ACTION


In [154]:
# Frame-wise prediction logic
def get_framewise_signal(symbol):
    decisions = {}
    for interval in intervals:
        model_path = f"{model_dir}/{symbol}_model_{interval}.pkl"
        if not os.path.exists(model_path):
            decisions[interval] = "❌ No model"
            continue

        model = joblib.load(model_path)
        latest_features = load_latest_features(symbol, interval)
        pred = model.predict(latest_features)[0]

        signal_map = {0: "📉 PUT", 1: "⏸️ NO ACTION", 2: "📈 CALL"}
        decisions[interval] = signal_map.get(pred, "❓ Unknown")
    
    return decisions

# Usage
nifty_signals = get_framewise_signal("NIFTY")
banknifty_signals = get_framewise_signal("BANKNIFTY")

print("\n🧭 Frame-wise Decision for NIFTY:")
for interval, signal in nifty_signals.items():
    print(f"  ⏱️ {interval}: {signal}")

print("\n🧭 Frame-wise Decision for BANKNIFTY:")
for interval, signal in banknifty_signals.items():
    print(f"  ⏱️ {interval}: {signal}")
# Save final decisions to CSV for UI to read
import os
os.makedirs("final_predictions", exist_ok=True)

for symbol in ["NIFTY", "BANKNIFTY"]:
    frame_signals = get_framewise_signal(symbol)
    df = pd.DataFrame.from_dict(frame_signals, orient="index", columns=["signal"])
    df.to_csv(f"final_predictions/{symbol}_framewise_predictions.csv")



🧭 Frame-wise Decision for NIFTY:
  ⏱️ 5m: ⏸️ NO ACTION
  ⏱️ 15m: ⏸️ NO ACTION
  ⏱️ 30m: ⏸️ NO ACTION
  ⏱️ 60m: ⏸️ NO ACTION

🧭 Frame-wise Decision for BANKNIFTY:
  ⏱️ 5m: ⏸️ NO ACTION
  ⏱️ 15m: ⏸️ NO ACTION
  ⏱️ 30m: 📉 PUT
  ⏱️ 60m: ⏸️ NO ACTION
